(page:advanced-fitting)=
# Advanced fitting for hyperfine beat (stage 1 bootstrap)

For advanced fitting, try a stage 1 style bootstrap. In this case, options are:

- "basic" ignore the photoionization dynamics and just try fitting the beat to the $l=4$, ROI=0 case, since it is already pretty close and may be assumed to be directly mapped here. See {ref}`page:advanced-fitting`.
- "advanced" set (arbitrary) parameters per final state for the probe, and fit these plus the hyperfine beat model parameters. This should allow for a match to a single set of hyperfine parameters for all observables, and fulfil the stage 1 bootstrap criteria. (This page.)

---

From prior work and data:

- Forbes, R. et al. (2018) ‘Quantum-beat photoelectron-imaging spectroscopy of Xe in the VUV’, Physical Review A, 97(6), p. 063417. Available at: https://doi.org/10.1103/PhysRevA.97.063417. arXiv: http://arxiv.org/abs/1803.01081, Authorea (original HTML version): https://doi.org/10.22541/au.156045380.07795038
- Data (OSF): https://osf.io/ds8mk/
- [Quantum Metrology with Photoelectrons (Github repo)](https://github.com/phockett/Quantum-Metrology-with-Photoelectrons), particularly the [Alignment 3 notebook](https://github.com/phockett/Quantum-Metrology-with-Photoelectrons/blob/master/Alignment/Alignment-3.ipynb). Functions from this notebook have been incorporated in the current project, under `qbanalysis.hyperfine`.

## Setup fitting model

Follow the modelling notebook ({ref}`page:hyperfine-beat-model`), but wrap functions for fitting.

New functions are in `qbanalysis.adv_fitting.py`.

### Imports

In [1]:
# Load packages
# Main functions used herein from qbanalysis.hyperfine
from qbanalysis.hyperfine import *
import numpy as np
from epsproc.sphCalc import setBLMs

from pathlib import Path

dataPath = Path('/tmp/xe_analysis')
# dataTypes = ['BLMall', 'BLMerr', 'BLMerrCycle']   # Read these types, should just do dir scan here.

# # Read from HDF5/NetCDF files
# # TO FIX: this should be identical to loadFinalDataset(dataPath), but gives slightly different plots - possibly complex/real/abs confusion?
# dataDict = {}
# for item in dataTypes:
#     dataDict[item] = IO.readXarray(fileName=f'Xe_dataset_{item}.nc', filePath=dataPath.as_posix()).real
#     dataDict[item].name = item

# Read from raw data files
from qbanalysis.dataset import loadFinalDataset
dataDict = loadFinalDataset(dataPath)

# Use Pandas and load Xe local data (ODS)
# These values were detemermined from the experimental data as detailed in ref. [4].
from qbanalysis.dataset import loadXeProps
xeProps = loadXeProps()

2024-06-19 12:33:30.621 | INFO     | qbanalysis.config:<module>:11 - PROJ_ROOT path is: /home/jovyan/code-share/github-share/Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


* sparse not found, sparse matrix forms not available. 
* natsort not found, some sorting functions not available. 


* Setting plotter defaults with epsproc.basicPlotters.setPlotters(). Run directly to modify, or change options in local env.


* Set Holoviews with bokeh.
* pyevtk not found, VTK export not available. 
2024-06-19 12:33:36.262 | INFO     | qbanalysis.hyperfine:<module>:28 - Using uncertainties modules, Sympy maths functions will be forced to float outputs.
2024-06-19 12:33:36.329 | INFO     | qbanalysis.dataset:loadDataset:244 - Loaded data cpBasex_results_cycleSummed_rot90_quad1_ROI_results_with_FT_NFFT1024_hanningWindow_270717.mat.
2024-06-19 12:33:36.373 | INFO     | qbanalysis.dataset:loadDataset:244 - Loaded data cpBasex_results_allCycles_ROIs_with_FTs_NFFT1024_hanningWindow_270717.mat.
2024-06-19 12:33:36.682 | INFO     | qbanalysis.dataset:loadFinalDataset:220 - Processed data to Xarray OK.
2024-06-19 12:33:36.721 | INFO     | qbanalysis.dataset:loadXeProps:71 - Loaded Xe data from /home/jovyan/code-share/github-share/Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV/dataLocal/Xe_data_table_fixedFractions.ods.

**Xe measured level splittings and the hyperfine constants.**
Statistical uncer

A/MHz      B/MHz   Splitting/cm−1
Isotope I   F   F′                                        
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan  0.2863+/-0.0005
131     1.5 1.5 0.5  1697+/-30     -8+/-7  0.0855+/-0.0010
            2.5 1.5  1697+/-30     -8+/-7  0.1411+/-0.0029
                0.5  1697+/-30     -8+/-7  0.2276+/-0.0029

## Improved model

- Add exponential state decays + error function (or masking) at t0.
- Add photoionization phenomenologically.

To test:

- trange.
- type of fit.
- adding noise.

In [2]:
# v2 pkg
from qbanalysis.adv_fitting import * 

2024-06-19 12:33:36.782 | INFO     | qbanalysis.basic_fitting:<module>:21 - Using uncertainties modules, Sympy maths functions will be forced to float outputs.
2024-06-19 12:33:36.782 | INFO     | qbanalysis.adv_fitting:<module>:26 - Using uncertainties modules, Sympy maths functions will be forced to float outputs.


In [8]:
# Create model parameters
params = initParams(xeProps)
params

name,value,initial value,min,max,vary
s0,0.28630000,0.2863,0.00000000,1.00000000,True
s1,0.08550000,0.0855,0.00000000,1.00000000,True
s2,0.14110000,0.1411,0.00000000,1.00000000,True
s3,0.22760000,0.2276,0.00000000,1.00000000,True
tau129,500.000000,500,-inf,inf,True
tau131,500.000000,500,-inf,inf,True
l2_amp_0,-0.41099899,-0.4109989864721906,-inf,inf,True
l4_amp_0,-0.22181471,-0.22181470690832272,-inf,inf,True
l2_offset_0,0.60479808,0.6047980797323451,-inf,inf,True
l4_offset_0,-0.14953117,-0.1495311713587888,-inf,inf,True


In [9]:
# Compute advanced model...
calcDict = calcAdvFitModel(params, xePropsFit=xeProps, dataDict=dataDict)

calcDict.keys()

dict_keys(['xData', 'xePropsFit', 'dataDict', 'trange', 'fitFlag', 'returnType', 'modelDict', 'modelDictSum', 'modelDA', 'modelSum', 'dataIn', 'modelIn', 'res', 'iso', 'dataCol', 'decay', 'ionization'])

In [10]:
# xr.concat([decay, decay.sum("Isotope").expand_dims({"Isotope":['sum']})], dim="Isotope")

In [11]:
# Plot model + decay
plotHyperfineModel(calcDict['modelDA']) * plotHyperfineModel(calcDict['decay'])

:Overlay
   .NdOverlay.I   :NdOverlay   [K,Q,Isotope]
      :Spread   [t]   (sum,sum_std)
   .NdOverlay.II  :NdOverlay   [K,Q,Isotope]
      :Curve   [t]   (sum,sum_std)
   .NdOverlay.III :NdOverlay   [K,Q,Isotope]
      :Spread   [t]   (decay,decay_std)
   .NdOverlay.IV  :NdOverlay   [K,Q,Isotope]
      :Curve   [t]   (decay,decay_std)

In [12]:
plotHyperfineModel(calcDict['ionization'],overlay=['l'])

:HoloMap   [ROI,Q]
   :Overlay
      .NdOverlay.I  :NdOverlay   [l]
         :Spread   [t]   (Ionization test,Ionization test_std)
      .NdOverlay.II :NdOverlay   [l]
         :Curve   [t]   (Ionization test,Ionization test_std)

In [22]:
modelDS = splitUncertaintiesToDataset(calcDict['ionization'].squeeze())
modelDS

dataDS = splitUncertaintiesToDataset(dataIn.squeeze())
dataDS

<xarray.Dataset>
Dimensions:          (ROI: 2, t: 97, l: 2)
Coordinates:
  * ROI              (ROI) int64 0 1
  * t                (t) float64 -70.0 -60.0 -50.0 -40.0 ... 870.0 880.0 890.0
  * l                (l) int64 2 4
    m                int64 0
Data variables:
    BLM input 0      (ROI, t, l) float64 0.09006 -0.05167 ... 0.6051 0.2423
    BLM input 0_std  (ROI, t, l) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

In [74]:
# Calc residuals...

# Basic function
def residual(model,dataIn):
    """
    Calc least squares residual
    """
    res = (model - dataIn)**2  # Returning single value XR only in testing? Issue with dims?
                            # Ah, OK after fixing t-units
    # res = model.values - dataIn.values  # Force to NP, assumes matching size.

    return res


def residualAdv(model,dataIn, dataUn = None):
    
    model.name = 'model'
    modelDS = splitUncertaintiesToDataset(model)
    modelDS = modelDS.assign_coords({'t':modelDS['t'].values.astype(int)})  # Force coords to avoid float comparison issues.
    
    dataIn.name = 'data'
    dataDS = splitUncertaintiesToDataset(dataIn)
    dataDS = dataDS.assign_coords({'t':dataDS['t'].values.astype(int)})
    
    # Uncertainties - use data_std if present, skip if zero.
    # Use passed data if provided
    res = (dataDS['data'] - modelDS['model'])
    thres = 1e-10
    if (dataUn is None) and (dataDS['data_std'].max() < thres):
        pass
    elif dataDS['data_std'].max() > thres:
        res = res/dataDS['data_std']
    else:
        res = res/dataUn
        
    res.name = 'res'
    
    return res, dataDS, modelDS
    # return (dataDS-model) / uncertainty

# dataIn = dataDict['BLMall'].sel({'ROI':0,'l':4}).copy()
dataIn = dataDict['BLMall'].unstack().sel({'l':[2,4]}).copy()
dataUn = dataDict['BLMerr'].unstack().sel({'l':[2,4]}).copy()  # Main data has uncertainties separately currently

# if trange is not None:
#     modelIn = modelIn.sel(t=slice(trange[0],trange[1]))
#     dataIn = dataIn.sel(t=slice(trange[0],trange[1]))

res = residual(calcDict['ionization'].squeeze(), dataIn.squeeze())
res.name = 'residual'


res, dataDS, modelDS = residualAdv(calcDict['ionization'].squeeze(), dataIn.squeeze(), dataUn = dataUn)

In [76]:
dataDS

<xarray.Dataset>
Dimensions:   (ROI: 2, t: 97, l: 2)
Coordinates:
  * ROI       (ROI) int64 0 1
  * t         (t) int64 -70 -60 -50 -40 -30 -20 -10 ... 840 850 860 870 880 890
  * l         (l) int64 2 4
    m         int64 0
Data variables:
    data      (ROI, t, l) float64 0.09006 -0.05167 0.0858 ... 0.6051 0.2423
    data_std  (ROI, t, l) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [77]:
# dataDS.assign_coords({'t':dataDS['t'].values.astype(int)})

In [78]:
# dataDS['data'] - modelDS['model']

In [79]:
res

<xarray.DataArray 'res' (ROI: 2, t: 97, l: 2, m: 1)>
array([[[[-8.81241476e+00],
         [ 2.69470665e+00]],

        [[-9.85257356e+00],
         [ 1.44400366e+00]],

        [[-9.96765026e+00],
         [ 2.33779816e+00]],

        [[-1.14365358e+01],
         [ 2.77227868e+00]],

        [[-9.13964767e+00],
         [ 3.83400226e-01]],

        [[-1.45445399e+01],
         [ 3.34940651e+00]],

        [[-1.43560199e+01],
         [ 3.47370952e+00]],
...
        [[ 6.32420076e+00],
         [ 1.44731820e-02]],

        [[ 6.83003725e+00],
         [ 1.15002351e+00]],

        [[ 1.02882111e+01],
         [-1.26175272e-01]],

        [[ 6.40624782e+00],
         [ 1.65558170e-01]],

        [[ 6.32487006e+00],
         [ 1.74393594e+00]],

        [[ 7.41635636e+00],
         [ 1.37280055e+00]],

        [[ 9.59889269e+00],
         [ 8.18224533e-01]]]])
Coordinates:
  * ROI      (ROI) int64 0 1
  * t        (t) int64 -70 -60 -50 -40 -30 -20 -10 ... 840 850 860 870 880 890
  * l        (l) int64 2 4
  * m        (m) int64 0
    Isotope  <U5 'sum'
    Q        int64 0

In [80]:
plotHyperfineModel(res,overlay=['l'])

:HoloMap   [ROI,m]
   :Overlay
      .NdOverlay.I  :NdOverlay   [l]
         :Spread   [t]   (res,res_std)
      .NdOverlay.II :NdOverlay   [l]
         :Curve   [t]   (res,res_std)

In [ ]:
ionTest = ionizationPhenom(params.valuesdict(),decay)
plotHyperfineModel(ionTest, overlay=['l'])

In [459]:
# # Functions for calculation.
# # NOTE: designed for use with lmfit, expect lmfit Parameters() or params dict.

# import lmfit
# from qbanalysis.basic_fitting import calcBasicFitModel


# def calcDecays(paramDict, isoDA):
#     """
#     Apply exponential decays, exp(-t/tau), per isotope.
    
#     paramDict : dict
#         Parameters dictionary with items 'tauZ' per isotope Z.
        
#     isoDA : xr.dataarray
#         Main data structure, with dims including "Isotope" and "t".
#         Groupby & apply exponential decay per Isotope.
        
#     """
    
#     #*** Apply exponential decay (per isotope)
#     # Easier way to do this...? Here set DA per isotope, then combine and multiply.
#     # Quick test of multiply without all isotope dims also awkward, although maybe groupby would work...
    
#     # Dict + loop version
#     # decay = xr.zeros_like(isoDA)
# #     decayDict = {}
# #     for iso in [129,131]:
# #         decayDict[iso] = np.exp(-isoDA.t/paramDict[f"tau{iso}"]).expand_dims({'Isotope':[f'{iso}Xe']})
# #         # decay = decay + np.exp(-isoDA.t/paramDict[f"tau{iso}"]).expand_dims({'Isotope':[f'{iso}Xe']})
    
# #     decayDA = isoDA*stackModelToDA(decayDict)
    
#     # Groupby version - better...?
#     # Test groupby...
#     decay = isoDA.groupby("Isotope").map(lambda x: x*np.exp(-x.t/paramDict[f"tau{x.Isotope.values.item().rstrip('Xe')}"]))
#     decay.name = 'decay'
    
#     return decay

# def ionizationPhenom(paramDict,modelDA):
#     """
#     Basic amplitude/phase + offset ionization channel model.

#     """
    
#     # Assume isotope independent...?
#     # Also ignore total XS here
#     modelIn = modelDA.sum("Isotope").sel({'K':2})
    
#     # ROI only version - assume isotope independent params
#     modelOutComponents = []
#     for ROI in [0,1]:
#         lparams = [2,4]
#         paramsAmp = [paramDict[f"l{l}_amp_{ROI}"] for l in lparams]
#         paramsOffset = [paramDict[f"l{l}_offset_{ROI}"] for l in lparams]
        
#         modelOutComponents.append((modelIn * xr.DataArray(paramsAmp,coords=[("l",lparams)]) + xr.DataArray(paramsOffset,coords=[("l",lparams)])).expand_dims({"ROI":[ROI]}))
        
#         # for item in ['amp','offset']:
#         #     [params.add(f"l{l}_{item}_{ROI}", value=1.0) for l in [2,4]]
    
#     modelOut = xr.concat(modelOutComponents,dim="ROI")
#     modelOut.name = "Ionization test"
    
#     # K2t = decay.sel({'K':2,'Isotope':'129Xe'}).squeeze()
#     # K2t = K2t * xr.DataArray([20,-10],coords=[("l",[0,2])]) + xr.DataArray([1,-1],coords=[("l",[0,2])])
#     # K2t.name = "Ionization test"

#     return modelOut
                                           

# def calcAdvFitModel(params, **kwargs):
#     """
#     Wrap basic fit model, and add some features.
    
#     Note passed params expect lmfit Parameters() object, or dictionary.
    
#     """
    
#     if isinstance(params, lmfit.parameter.Parameters):
#         paramDict = params.valuesdict()
#     else:
#         paramDict = params
    
#     #*** Run basic case as base
#     # TODO: return modelDA here too
#     # TODO: arg passing
#     # xDataBasic = xData[0:4]  # For basic case only use first 4 (splitting) params
#     # xDataBasic = [paramDict[k] for k,v in paramDict.items() if k.startswith('s')]  # For dict case, use s0...3
#     xDataBasic = [paramDict[f"s{n}"] for n in range(0,4)]  # Assume number of params and enforce ordering. 
#     calcBasicDict = calcBasicFitModel(xDataBasic, fitFlag=False, returnType='full', **kwargs)
    
#     # Use original model results and apply additional params
#     # modelDA = stackModelToDA(calcDict['modelDict'])  # Use original model?
#     isoDA = calcBasicDict['modelDA'].sel({'Isotope':['129Xe','131Xe']})  # Use isotope-weighted results
    
    
#     #*** Apply exponential decay (per isotope)
#     # Easier way to do this...? Here set DA per isotope, then combine and multiply.
#     # Quick test of multiply without all isotope dims also awkward, although maybe groupby would work...
    
#     # Dict + loop version
#     # decay = xr.zeros_like(isoDA)
# #     decayDict = {}
# #     for iso in [129,131]:
# #         decayDict[iso] = np.exp(-isoDA.t/paramDict[f"tau{iso}"]).expand_dims({'Isotope':[f'{iso}Xe']})
# #         # decay = decay + np.exp(-isoDA.t/paramDict[f"tau{iso}"]).expand_dims({'Isotope':[f'{iso}Xe']})
    
# #     decayDA = isoDA*stackModelToDA(decayDict)
    
#     # Groupby version - better...?
#     # Test groupby...
#     # decay = isoDA.groupby("Isotope").map(lambda x: x*np.exp(-x.t/paramDict[f"tau{x.Isotope.values.item().rstrip('Xe')}"]))
#     # decay.name = 'decay'
    
#     decay = calcDecays(paramDict, isoDA)

#     #*** Apply ionization model
#     # 
    
#     return calcBasicDict, decay
    

In [460]:
plotOpts = {'width':800}
fitParamsCol = 'Splitting/cm−1'
xePropsFit = xeProps.copy()
x0 = unumpy.nominal_values(xePropsFit[fitParamsCol].to_numpy())

# calcDict = calcAdvFitModel(x0, xePropsFit=xePropsFit, dataDict=dataDict)
calcDict, decay = calcAdvFitModel(params, xePropsFit=xePropsFit, dataDict=dataDict)


In [461]:
calcDict.keys()

dict_keys(['xData', 'xePropsFit', 'dataDict', 'trange', 'fitFlag', 'returnType', 'modelDict', 'modelDictSum', 'modelDA', 'modelSum', 'dataIn', 'modelIn', 'res', 'iso', 'dataCol'])

In [462]:
decay

<xarray.DataArray 'decay' (Isotope: 2, t: 97, TKQ: 2)>
array([[[0.058443086314030265+/-1.8152364255927824e-05,
         0.016860756473894418+/-5.2369341259643426e-06],
        [0.057285835668964655+/-1.779292336104142e-05,
         0.026765626419518664+/-8.313376841437429e-06],
        [0.056151500121987954+/-1.7440599872741574e-05,
         0.021310973914671946+/-6.6191672196961415e-06],
        [0.055039625923756705+/-1.7095252856935255e-05,
         0.0027676311383275382+/-8.596234681895795e-07],
        [0.053949768309766416+/-1.6756744170211456e-05,
         -0.023038732269251586+/-7.155807239527246e-06],
        [0.05288149132243987+/-1.6424938404581974e-05,
         -0.048490109799216335+/-1.5060979688097011e-05],
        [0.05183436763673825+/-1.6099702833316424e-05,
         -0.06636893936964679+/-2.0614126301337982e-05],
        [0.05080797838922545+/-1.578090735784977e-05,
         -0.07185333211480174+/-2.2317573212024507e-05],
        [0.049801913010517077+/-1.5468424455741156e-05,
         -0.06376657603409748+/-1.9805834847677677e-05],
        [0.04881576906104727+/-1.516212912966325e-05,
         -0.044762012993739386+/-1.3903036542679444e-05],
...
        [0.008249849657868177+/-1.1656500901266237e-06,
         0.004239923951173211+/-5.990736729488722e-07],
        [0.008086491689622576+/-1.1425686718820165e-06,
         0.004796873923938971+/-6.777670810561646e-07],
        [0.007926368425874152+/-1.1199442963406142e-06,
         0.004419340536115919+/-6.244240692690301e-07],
        [0.007769415815182399+/-1.0977679134505375e-06,
         0.003240838962956469+/-4.57909463314058e-07],
        [0.007615571074410309+/-1.0760306523629417e-06,
         0.001496735961535446+/-2.1147905486927232e-07],
        [0.007464772663610286+/-1.054723817883558e-06,
         -0.0005188357091932647+/-7.330811060359611e-08],
        [0.007316960261407336+/-1.0338388869944992e-06,
         -0.0025026249705191305+/-3.536046283772626e-07],
        [0.00717207474086972+/-1.0133675054449408e-06,
         -0.0041898307990522856+/-5.919958364177794e-07],
        [0.007030058145857391+/-9.933014844093072e-07,
         -0.00539176436736191+/-7.618212308587691e-07],
        [0.006890853667838762+/-9.736327972116335e-07,
         -0.006017909266194136+/-8.50291431895707e-07]]], dtype=object)
Coordinates:
  * Isotope  (Isotope) <U5 '129Xe' '131Xe'
  * t        (t) float64 -70.0 -60.0 -50.0 -40.0 ... 860.0 870.0 880.0 890.0
  * TKQ      (TKQ) MultiIndex
  - K        (TKQ) int64 0 2
  - Q        (TKQ) int64 0 0

In [463]:
# Ionization... just want amplitude + offset per channel...?

# decay.sel({'K':2}).groupby('Isotope').apply(

# Test single channel... OK
K2t = decay.sel({'K':2,'Isotope':'129Xe'}).squeeze()
K2t = K2t * xr.DataArray([20,-10],coords=[("l",[0,2])]) + xr.DataArray([1,-1],coords=[("l",[0,2])])
K2t.name = "Ionization test"

dataPlot = K2t
overlay = 'l'

DS = splitUncertaintiesToDataset(dataPlot)
hvDS = hvPlotters.hv.Dataset(DS.unstack())
hvDS.to(hvPlotters.hv.Curve, kdims = ['t']).overlay(overlay)
# plotHyperfineModel(K2t, plotSpread=False, overlay=['l'])  # NOTE - now debugged!

:NdOverlay   [l]
   :Curve   [t]   (Ionization test,Ionization test_std)

In [464]:
plotHyperfineModel(isoDA) * plotHyperfineModel(decay.sum('Isotope'))

:Overlay
   .NdOverlay.I   :NdOverlay   [K,Q,Isotope]
      :Spread   [t]   (sum,sum_std)
   .NdOverlay.II  :NdOverlay   [K,Q,Isotope]
      :Curve   [t]   (sum,sum_std)
   .NdOverlay.III :NdOverlay   [K,Q]
      :Spread   [t]   (decay,decay_std)
   .NdOverlay.IV  :NdOverlay   [K,Q]
      :Curve   [t]   (decay,decay_std)

In [467]:
ionTest = ionizationPhenom(params.valuesdict(),decay)
plotHyperfineModel(ionTest, overlay=['l'])

:HoloMap   [ROI,Q]
   :Overlay
      .NdOverlay.I  :NdOverlay   [l]
         :Spread   [t]   (Ionization test,Ionization test_std)
      .NdOverlay.II :NdOverlay   [l]
         :Curve   [t]   (Ionization test,Ionization test_std)

In [468]:
ionTest

<xarray.DataArray 'Ionization test' (ROI: 2, t: 97, Q: 1, l: 2)>
array([[[[0.6403781128075106+/-3.83585990074327e-06,
          -0.3936582375815621+/-2.114092929308782e-06]],

        [[0.6408697244723353+/-5.849303958153424e-06,
          -0.39392918406330674+/-3.2237809667954462e-06]],

        [[0.6539562713550541+/-4.833866627949529e-06,
          -0.40114169363083885+/-2.6641336033648176e-06]],

        [[0.6768522086355124+/-2.7535565265556446e-06,
          -0.41376054321529193+/-1.5175930648862734e-06]],

        [[0.7041523242421684+/-6.416768260332067e-06,
          -0.4288067083169319+/-3.5365328138163846e-06]],

        [[0.7292763485142447+/-1.171631153508789e-05,
          -0.44265354408355806+/-6.457319092678202e-06]],

        [[0.7462004468325518+/-1.5551819051342283e-05,
          -0.4519810788516362+/-8.571217809065863e-06]],
...
        [[-0.3076034610579288+/-1.439956609950813e-06,
          0.8974014263551447+/-1.8133286862799615e-06]],

        [[-0.3086845089920896+/-8.704317418987261e-07,
          0.8987627835076526+/-1.0961294501002437e-06]],

        [[-0.30984139536491523+/-2.4787966870034143e-07,
          0.9002196435492081+/-3.1215337385426926e-07]],

        [[-0.31064706097148664+/-2.994678997493439e-07,
          0.9012342133951668+/-3.7711812250651527e-07]],

        [[-0.3107814766605829+/-5.761008126629101e-07,
          0.9014034822628814+/-7.254802836222525e-07]],

        [[-0.31013102819947574+/-5.704274625128601e-07,
          0.9005843764333573+/-7.18335867947986e-07]],

        [[-0.3088235436073565+/-3.8408510102688807e-07,
          0.8989378689612746+/-4.836760544392958e-07]]]], dtype=object)
Coordinates:
  * ROI      (ROI) int64 0 1
  * t        (t) float64 -70.0 -60.0 -50.0 -40.0 ... 860.0 870.0 880.0 890.0
  * Q        (Q) int64 0
  * l        (l) int64 2 4

In [438]:
decay

<xarray.DataArray 'decay' (Isotope: 2, t: 97, TKQ: 2)>
array([[[0.058443086314030265+/-1.8152364255927824e-05,
         0.016860756473894418+/-5.2369341259643426e-06],
        [0.057285835668964655+/-1.779292336104142e-05,
         0.026765626419518664+/-8.313376841437429e-06],
        [0.056151500121987954+/-1.7440599872741574e-05,
         0.021310973914671946+/-6.6191672196961415e-06],
        [0.055039625923756705+/-1.7095252856935255e-05,
         0.0027676311383275382+/-8.596234681895795e-07],
        [0.053949768309766416+/-1.6756744170211456e-05,
         -0.023038732269251586+/-7.155807239527246e-06],
        [0.05288149132243987+/-1.6424938404581974e-05,
         -0.048490109799216335+/-1.5060979688097011e-05],
        [0.05183436763673825+/-1.6099702833316424e-05,
         -0.06636893936964679+/-2.0614126301337982e-05],
        [0.05080797838922545+/-1.578090735784977e-05,
         -0.07185333211480174+/-2.2317573212024507e-05],
        [0.049801913010517077+/-1.5468424455741156e-05,
         -0.06376657603409748+/-1.9805834847677677e-05],
        [0.04881576906104727+/-1.516212912966325e-05,
         -0.044762012993739386+/-1.3903036542679444e-05],
...
        [0.008249849657868177+/-1.1656500901266237e-06,
         0.004239923951173211+/-5.990736729488722e-07],
        [0.008086491689622576+/-1.1425686718820165e-06,
         0.004796873923938971+/-6.777670810561646e-07],
        [0.007926368425874152+/-1.1199442963406142e-06,
         0.004419340536115919+/-6.244240692690301e-07],
        [0.007769415815182399+/-1.0977679134505375e-06,
         0.003240838962956469+/-4.57909463314058e-07],
        [0.007615571074410309+/-1.0760306523629417e-06,
         0.001496735961535446+/-2.1147905486927232e-07],
        [0.007464772663610286+/-1.054723817883558e-06,
         -0.0005188357091932647+/-7.330811060359611e-08],
        [0.007316960261407336+/-1.0338388869944992e-06,
         -0.0025026249705191305+/-3.536046283772626e-07],
        [0.00717207474086972+/-1.0133675054449408e-06,
         -0.0041898307990522856+/-5.919958364177794e-07],
        [0.007030058145857391+/-9.933014844093072e-07,
         -0.00539176436736191+/-7.618212308587691e-07],
        [0.006890853667838762+/-9.736327972116335e-07,
         -0.006017909266194136+/-8.50291431895707e-07]]], dtype=object)
Coordinates:
  * Isotope  (Isotope) <U5 '129Xe' '131Xe'
  * t        (t) float64 -70.0 -60.0 -50.0 -40.0 ... 860.0 870.0 880.0 890.0
  * TKQ      (TKQ) MultiIndex
  - K        (TKQ) int64 0 2
  - Q        (TKQ) int64 0 0

In [429]:
ionizationPhenom(params.valuesdict(),decay)

<xarray.DataArray 'Ionization test' (t: 97, Q: 1, l: 2)>
array([[[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]],

...

       [[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]],

       [[3.0, 3.0]]], dtype=object)
Coordinates:
  * t        (t) float64 -70.0 -60.0 -50.0 -40.0 ... 860.0 870.0 880.0 890.0
  * Q        (Q) int64 0
  * l        (l) int64 2 4

In [424]:
isoDA = calcDict['modelDA'].sel({'Isotope':['129Xe','131Xe']})

In [425]:
isoDA

<xarray.DataArray 'sum' (Isotope: 2, t: 97, TKQ: 2)>
array([[[0.05080797838922545+/-1.578090735784977e-05,
         0.014658037495633842+/-4.552771810648148e-06],
        [0.05080797838922545+/-1.578090735784977e-05,
         0.02373898107300778+/-7.373303818801988e-06],
        [0.05080797838922545+/-1.578090735784977e-05,
         0.019282966612783448+/-5.989270176618117e-06],
        [0.05080797838922545+/-1.578090735784977e-05,
         0.0025548455445588026+/-7.93532475223373e-07],
        [0.05080797838922545+/-1.578090735784977e-05,
         -0.0216970609499241+/-6.739085467352166e-06],
        [0.05080797838922545+/-1.578090735784977e-05,
         -0.046588785398423635+/-1.4470430227611258e-05],
        [0.05080797838922545+/-1.578090735784977e-05,
         -0.06505474631890426+/-2.0205939251949383e-05],
        [0.05080797838922545+/-1.578090735784977e-05,
         -0.07185333211480174+/-2.2317573212024507e-05],
        [0.05080797838922545+/-1.578090735784977e-05,
         -0.06505474631890426+/-2.0205939251949383e-05],
        [0.05080797838922545+/-1.578090735784977e-05,
         -0.046588785398423635+/-1.4470430227611258e-05],
...
        [0.040861772851806026+/-5.773502691896256e-06,
         0.021000480807130368+/-2.9672313267172388e-06],
        [0.040861772851806026+/-5.773502691896256e-06,
         0.024239037174833736+/-3.424818274170664e-06],
        [0.040861772851806026+/-5.773502691896256e-06,
         0.022782449595967653+/-3.2190119246012207e-06],
        [0.040861772851806026+/-5.773502691896256e-06,
         0.017044579502982996+/-2.4082882061824844e-06],
        [0.040861772851806026+/-5.773502691896256e-06,
         0.008030820575609502+/-1.1347027056210559e-06],
        [0.040861772851806026+/-5.773502691896256e-06,
         -0.002840079377073403+/-4.01284740831004e-07],
        [0.040861772851806026+/-5.773502691896256e-06,
         -0.013975980383272104+/-1.974715112272579e-06],
        [0.040861772851806026+/-5.773502691896256e-06,
         -0.023870904945089075+/-3.372803584864039e-06],
        [0.040861772851806026+/-5.773502691896256e-06,
         -0.03133929283066178+/-4.428038210093316e-06],
        [0.040861772851806026+/-5.773502691896256e-06,
         -0.03568533788864044+/-5.042106105099815e-06]]], dtype=object)
Coordinates:
  * Isotope  (Isotope) <U5 '129Xe' '131Xe'
  * t        (t) float64 -70.0 -60.0 -50.0 -40.0 ... 860.0 870.0 880.0 890.0
  * TKQ      (TKQ) MultiIndex
  - K        (TKQ) int64 0 2
  - Q        (TKQ) int64 0 0
Attributes:
    data:        sum
    renormFlag:  True
    renorm:      0.47633+/-0.00009

In [426]:
decay = xr.ones_like(isoDA)

dTemp = {}

for iso in [129,131]:
    # # decay = decay.sel({"Isotope":f"{iso}Xe"}, drop=False) * np.exp(-isoDA.t/paramDict[f"tau{iso}"])

    dTemp[iso] = np.exp(-isoDA.t/paramDict[f"tau{iso}"]).expand_dims({'Isotope':[f'{iso}Xe']})
    
    

# decay.where(
# decay = decay*dTemp

# decay.sel({"Isotope":"129Xe"}) * dTemp + decay.sel({"Isotope":"131Xe"})
# decay


decayStack = stackModelToDA(dTemp)

isoDA*decayStack

<xarray.DataArray (Isotope: 2, t: 97, TKQ: 2)>
array([[[0.05152429258629374+/-1.6003393832246565e-05,
         0.014864693235384915+/-4.616958877076896e-06],
        [0.051421346981041616+/-1.5971419030042544e-05,
         0.024025564909907482+/-7.462316472399921e-06],
        [0.051318607061245955+/-1.5939508113535934e-05,
         0.019476763649120352+/-6.049463342605353e-06],
        [0.05121607241594699+/-1.5907660955083036e-05,
         0.002575366282422911+/-7.99906195914785e-07],
        [0.051113742635005954+/-1.5875877427295168e-05,
         -0.021827634644987985+/-6.779641526666116e-06],
        [0.05101161730910362+/-1.5844157403038174e-05,
         -0.046775513747744905+/-1.4528427866469255e-05],
        [0.050909696029738535+/-1.581250075543192e-05,
         -0.06518498600781776+/-2.0246391569286516e-05],
        [0.05080797838922545+/-1.578090735784977e-05,
         -0.07185333211480174+/-2.2317573212024507e-05],
        [0.050706463980693665+/-1.574937708391809e-05,
         -0.06492476684906279+/-2.0165567758396203e-05],
        [0.050605152398085405+/-1.571790980751575e-05,
         -0.046402802470662634+/-1.4412664115945605e-05],
...
        [0.03482010594181466+/-4.919854459479097e-06,
         0.01789542928510027+/-2.528507745488066e-06],
        [0.034750535323739305+/-4.9100245837125295e-06,
         0.02061387597186109+/-2.9126065784171013e-06],
        [0.03468110370785159+/-4.900214348050845e-06,
         0.019336422333465487+/-2.7321106893425357e-06],
        [0.03461181081642494+/-4.8904237132530856e-06,
         0.014437546881343575+/-2.039931455889618e-06],
        [0.034542656372287706+/-4.8806526401567e-06,
         0.006788884969255951+/-9.592252834238172e-07],
        [0.03447364009882202+/-4.870901089677383e-06,
         -0.0023960750467778044+/-3.385498172761164e-07],
        [0.03440476171996269+/-4.861169022808919e-06,
         -0.01176748440732662+/-1.6626689974746077e-06],
        [0.03433602096019612+/-4.851456400623027e-06,
         -0.020058647369657658+/-2.834156388772488e-06],
        [0.03426741754455917+/-4.841763184269207e-06,
         -0.026281694552859607+/-3.7134324738879644e-06],
        [0.03419895119863809+/-4.8320893349745795e-06,
         -0.029866573175534285+/-4.219952503089753e-06]]], dtype=object)
Coordinates:
  * Isotope  (Isotope) <U5 '129Xe' '131Xe'
  * t        (t) float64 -70.0 -60.0 -50.0 -40.0 ... 860.0 870.0 880.0 890.0
  * TKQ      (TKQ) MultiIndex
  - K        (TKQ) int64 0 2
  - Q        (TKQ) int64 0 0

In [334]:
isoDA.Isotope.values.item()  #.rstrip('Xe')

ValueError: can only convert an array of size 1 to a Python scalar

In [336]:
# Test groupby...
isoDA.groupby("Isotope").map(lambda x: np.exp(-x.t/paramDict[f"tau{x.Isotope.values.item().rstrip('Xe')}"]))
# isoDA.groupby("Isotope").map(lambda x: print(x.Isotope.values.item().rstrip('Xe')))

<xarray.DataArray 't' (Isotope: 2, t: 97)>
array([[1.01409846, 1.01207229, 1.01005017, 1.00803209, 1.00601804,
        1.00400801, 1.002002  , 1.        , 0.998002  , 0.99600799,
        0.99401796, 0.99203191, 0.99004983, 0.98807171, 0.98609754,
        0.98412732, 0.98216103, 0.98019867, 0.97824024, 0.97628571,
        0.97433509, 0.97238837, 0.97044553, 0.96850658, 0.9665715 ,
        0.96464029, 0.96271294, 0.96078944, 0.95886978, 0.95695396,
        0.95504196, 0.95313379, 0.95122942, 0.94932887, 0.94743211,
        0.94553914, 0.94364995, 0.94176453, 0.93988289, 0.938005  ,
        0.93613086, 0.93426047, 0.93239382, 0.9305309 , 0.92867169,
        0.92681621, 0.92496443, 0.92311635, 0.92127196, 0.91943126,
        0.91759423, 0.91576088, 0.91393119, 0.91210515, 0.91028276,
        0.90846402, 0.9066489 , 0.90483742, 0.90302955, 0.9012253 ,
        0.89942465, 0.8976276 , 0.89583414, 0.89404426, 0.89225796,
        0.89047522, 0.88869605, 0.88692044, 0.88514837, 0.88337984,
        0.88161485, 0.87985338, 0.87809543, 0.876341  , 0.87459006,
        0.87284263, 0.87109869, 0.86935824, 0.86762126, 0.86588775,
        0.8641577 , 0.86243111, 0.86070798, 0.85898828, 0.85727202,
        0.85555919, 0.85384978, 0.85214379, 0.8504412 , 0.84874202,
        0.84704623, 0.84535383, 0.84366482, 0.84197917, 0.8402969 ,
        0.83861798, 0.83694242],
       [1.01409846, 1.01207229, 1.01005017, 1.00803209, 1.00601804,
        1.00400801, 1.002002  , 1.        , 0.998002  , 0.99600799,
        0.99401796, 0.99203191, 0.99004983, 0.98807171, 0.98609754,
        0.98412732, 0.98216103, 0.98019867, 0.97824024, 0.97628571,
        0.97433509, 0.97238837, 0.97044553, 0.96850658, 0.9665715 ,
        0.96464029, 0.96271294, 0.96078944, 0.95886978, 0.95695396,
        0.95504196, 0.95313379, 0.95122942, 0.94932887, 0.94743211,
        0.94553914, 0.94364995, 0.94176453, 0.93988289, 0.938005  ,
        0.93613086, 0.93426047, 0.93239382, 0.9305309 , 0.92867169,
        0.92681621, 0.92496443, 0.92311635, 0.92127196, 0.91943126,
        0.91759423, 0.91576088, 0.91393119, 0.91210515, 0.91028276,
        0.90846402, 0.9066489 , 0.90483742, 0.90302955, 0.9012253 ,
        0.89942465, 0.8976276 , 0.89583414, 0.89404426, 0.89225796,
        0.89047522, 0.88869605, 0.88692044, 0.88514837, 0.88337984,
        0.88161485, 0.87985338, 0.87809543, 0.876341  , 0.87459006,
        0.87284263, 0.87109869, 0.86935824, 0.86762126, 0.86588775,
        0.8641577 , 0.86243111, 0.86070798, 0.85898828, 0.85727202,
        0.85555919, 0.85384978, 0.85214379, 0.8504412 , 0.84874202,
        0.84704623, 0.84535383, 0.84366482, 0.84197917, 0.8402969 ,
        0.83861798, 0.83694242]])
Coordinates:
  * Isotope  (Isotope) <U5 '129Xe' '131Xe'
  * t        (t) float64 -70.0 -60.0 -50.0 -40.0 ... 860.0 870.0 880.0 890.0

In [343]:
from scipy.special import erf  # Error function if required, or use as mask?

tau=500
# decay = erf(isoDA.t)*np.exp(-isoDA.t/tau).expand_dims({'Isotope':['129Xe']})  # Just add isotope dim to allow * by main data?
decay = np.exp(-isoDA.t/tau).expand_dims({'Isotope':['129Xe']})  # Just add isotope dim to allow * by main data?


In [344]:
# Basics OK.
# TODO: should multiply only K=0 and renorm?
# TODO: remove t<=0 terms.
testMult = isoDA * decay

testMult.values = unumpy.nominal_values(testMult)

# xr.where(testMult.t>0, unumpy.nominal_values(testMult),0)

testMult.squeeze().unstack().hvplot.line(x='t')

:DynamicMap   [Q,K]
   :Curve   [t]   (value)

In [215]:
erf(isoDA.t)

<xarray.DataArray 't' (t: 97)>
array([-1., -1., -1., -1., -1., -1., -1.,  0.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.])
Coordinates:
  * t        (t) float64 -70.0 -60.0 -50.0 -40.0 ... 860.0 870.0 880.0 890.0
Attributes:
    units:    ps

## Fitting with lmfit

- Easy parameter control.
- Propagate uncertainties.

See https://lmfit.github.io/lmfit-py/intro.html

In [27]:
xeProps

A/MHz      B/MHz   Splitting/cm−1
Isotope I   F   F′                                        
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan  0.2863+/-0.0005
131     1.5 1.5 0.5  1697+/-30     -8+/-7  0.0855+/-0.0010
            2.5 1.5  1697+/-30     -8+/-7  0.1411+/-0.0029
                0.5  1697+/-30     -8+/-7  0.2276+/-0.0029

In [185]:
# def pdIndexMap(df):
#     """
#     Convert list of tuple labels to short str format from PD dataframe.
    
#     Also append short names as column in df.
    
#     Useful for setting up mappings to fitting params for lmfit.
#     """
    
#     indList = df.index
    
#     # Parameter names for lmfit [a-z_][a-z0-9_]*, so replace '.' and '-' signs.
#     nameList = ['_'.join(str(ele).replace('-','n').replace('.','') for ele in sub) for sub in indList]
    
#     # shortNameList = [list(f"{str(ele)}_s{n}" for ele in sub) for n,sub in enumerate(indList)]
#     # shortNameList = [f"I{str(sub[0])}_s{n}" for n,sub in enumerate(indList)]
#     shortNameList = [f"s{n}" for n,sub in enumerate(indList)]
    
#     # Generate map {full names : lables}
#     indMapLong = dict(zip(indList, nameList))
#     indMapShort = dict(zip(indList, shortNameList))
    
#     # Append to original table
#     df['label']=shortNameList
    
#     return locals()


# def pdParamsReplaceFromMap(df, pdMap, params, dataCol = 'Splitting/cm−1'):
#     """
#     Convert lmfit.params items back to original PD dataframe via labels.
#     """

#     dfOut = df.copy()

#     for k,v in pdMap['indMapShort'].items():
#         # Replace by keys
#         # xeTest.loc[k][dataCol] = xeTest.loc[k][dataCol]*n

#         # Replace by value (short name) lookup
#         # This should ensure consistency in replacement ordering etc.
#         dfOut.loc[dfOut['label']==v,dataCol] = params.valuesdict()[v]
        
#         # print(params.valuesdict()[v])


#     return dfOut

In [186]:
pdMap = pdIndexMap(xeProps)
# xeProps['lmfit']=pdMap['shortNameList']
xeProps
# pdMap

A/MHz      B/MHz   Splitting/cm−1 shortName  \
Isotope I   F   F′                                                     
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan  0.2863+/-0.0005  0_I129_s   
131     1.5 1.5 0.5  1697+/-30     -8+/-7  0.0855+/-0.0010  1_I131_s   
            2.5 1.5  1697+/-30     -8+/-7  0.1411+/-0.0029  2_I131_s   
                0.5  1697+/-30     -8+/-7  0.2276+/-0.0029  3_I131_s   

                        lmfit label  
Isotope I   F   F′                   
129     0.5 0.5 1.5  0_I129_s    s0  
131     1.5 1.5 0.5  1_I131_s    s1  
            2.5 1.5  2_I131_s    s2  
                0.5  3_I131_s    s3

In [470]:
# Create parameters for the fit
# v2 better
# from lmfit import minimize, Parameters

# def initParams(xeProps):
#     """
#     Init lmfit Parameters() for Xe advanced hyperfine model
    
#     """
    
#     # Set labels for params
#     pdMap = pdIndexMap(xeProps)

#     # Setup parameters from df
#     params = Parameters()

#     # Iterate over PD rows and assign to params.
#     # May be neater way to do this...?
#     for item in xeProps.iterrows():
#         itemVal = unumpy.nominal_values(item[1]['Splitting/cm−1'])
#         params.add(item[1]['label'], value = itemVal, min = 0, max = 1)

#     #*** Add additional params as required for the advanced model...
#     # Lifetimes
#     params.add("tau129", value = 500)
#     params.add("tau131", value = 500)

#     # Ionization model params - just set amplitude + offset for l=2,4 modelling
#     # Will also need ROI (channel) here too...? Or just apply these to sum, if assumed iso independent
#     # for iso in [129,131]:
#     #     for item in ['amp','offset']:
#     #         [params.add(f"l{l}_{item}_{iso}", value=1.0) for l in [2,4]]

#     # ROI only version - assume isotope independent params
#     for ROI in [0,1]:
#         for item in ['amp','offset']:
#             [params.add(f"l{l}_{item}_{ROI}", value=np.random.uniform(-1,1)) for l in [2,4]]

#     return params

params = initParams(xeProps)
params

name,value,initial value,min,max,vary
s0,0.28630000,0.2863,0.00000000,1.00000000,True
s1,0.08550000,0.0855,0.00000000,1.00000000,True
s2,0.14110000,0.1411,0.00000000,1.00000000,True
s3,0.22760000,0.2276,0.00000000,1.00000000,True
tau129,500.000000,500,-inf,inf,True
tau131,500.000000,500,-inf,inf,True
l2_amp_0,0.92752109,0.927521091520729,-inf,inf,True
l4_amp_0,-0.72044186,-0.7204418641921053,-inf,inf,True
l2_offset_0,-0.73814177,-0.738141772756711,-inf,inf,True
l4_offset_0,0.98695291,0.9869529065417333,-inf,inf,True


In [405]:
dataDict

{'BLMall': <xarray.DataArray 'BLM input 0' (ROI: 2, BLM: 3, t: 97)>
 array([[[ 1.        ,  1.        ,  1.        ,  1.        ,
           1.        ,  1.        ,  1.        ,  1.        ,
           1.        ,  1.        ,  1.        ,  1.        ,
           1.        ,  1.        ,  1.        ,  1.        ,
           1.        ,  1.        ,  1.        ,  1.        ,
           1.        ,  1.        ,  1.        ,  1.        ,
           1.        ,  1.        ,  1.        ,  1.        ,
           1.        ,  1.        ,  1.        ,  1.        ,
           1.        ,  1.        ,  1.        ,  1.        ,
           1.        ,  1.        ,  1.        ,  1.        ,
           1.        ,  1.        ,  1.        ,  1.        ,
           1.        ,  1.        ,  1.        ,  1.        ,
           1.        ,  1.        ,  1.        ,  1.        ,
           1.        ,  1.        ,  1.        ,  1.        ,
           1.        ,  1.        ,  1.        ,  1.        ,
  

In [254]:
type(params)  #['s0']   #.valuesdict()['s0']

lmfit.parameter.Parameters

In [190]:
# Test param remapper
pdParamsReplaceFromMap(xeProps,pdMap, params)

0.2863
0.0855
0.1411
0.2276


A/MHz      B/MHz Splitting/cm−1 shortName     lmfit  \
Isotope I   F   F′                                                             
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan         0.2863  0_I129_s  0_I129_s   
131     1.5 1.5 0.5  1697+/-30     -8+/-7         0.0855  1_I131_s  1_I131_s   
            2.5 1.5  1697+/-30     -8+/-7         0.1411  2_I131_s  2_I131_s   
                0.5  1697+/-30     -8+/-7         0.2276  3_I131_s  3_I131_s   

                    label  
Isotope I   F   F′         
129     0.5 0.5 1.5    s0  
131     1.5 1.5 0.5    s1  
            2.5 1.5    s2  
                0.5    s3

In [176]:
params.valuesdict()

{'s0': 1, 's1': 0.8550000000000001, 's2': 1, 's3': 1}

In [264]:
# Check unpacking - ordering OK...?
paramDict = params.valuesdict()
[paramDict[k] for k,v in paramDict.items() if k.startswith('s')] 

[array(0.2863), array(0.0855), array(0.1411), array(0.2276)]

In [266]:
# Enforce...
[paramDict[f"s{n}"] for n in range(0,4)] 

[array(0.2863), array(0.0855), array(0.1411), array(0.2276)]

In [165]:
dataCol = 'Splitting/cm−1'
# [xeProps.loc[k][dataCol]*2 for k,v in pdMap['indMapShort'].items()]

xeTest = xeProps.copy()
n=1
for k,v in pdMap['indMapShort'].items():
    # Replace by keys
    # xeTest.loc[k][dataCol] = xeTest.loc[k][dataCol]*n
    
    # Replace by value (short name) lookup
    # This should ensure consistency in replacement
    xeTest.loc[xeTest['label']==v,dataCol] = params.valuesdict()[v]
    
    n = n+5
    
xeTest
    



A/MHz      B/MHz Splitting/cm−1 shortName     lmfit  \
Isotope I   F   F′                                                             
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan         0.2863  0_I129_s  0_I129_s   
131     1.5 1.5 0.5  1697+/-30     -8+/-7         0.0855  1_I131_s  1_I131_s   
            2.5 1.5  1697+/-30     -8+/-7         0.1411  2_I131_s  2_I131_s   
                0.5  1697+/-30     -8+/-7         0.2276  3_I131_s  3_I131_s   

                       label  
Isotope I   F   F′            
129     0.5 0.5 1.5  I129_0s  
131     1.5 1.5 0.5  I131_1s  
            2.5 1.5  I131_2s  
                0.5  I131_3s

In [158]:
xeProps

A/MHz      B/MHz   Splitting/cm−1 shortName  \
Isotope I   F   F′                                                     
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan  0.2863+/-0.0005  0_I129_s   
131     1.5 1.5 0.5  1697+/-30     -8+/-7  0.0855+/-0.0010  1_I131_s   
            2.5 1.5  1697+/-30     -8+/-7  0.1411+/-0.0029  2_I131_s   
                0.5  1697+/-30     -8+/-7  0.2276+/-0.0029  3_I131_s   

                        lmfit    label  
Isotope I   F   F′                      
129     0.5 0.5 1.5  0_I129_s  I129_0s  
131     1.5 1.5 0.5  1_I131_s  I131_1s  
            2.5 1.5  2_I131_s  I131_2s  
                0.5  3_I131_s  I131_3s

In [110]:
# Create parameters for the fit
# v1 testing
from lmfit import minimize, Parameters

params = Parameters()

# Iterate over PD rows and assign to params.
# May be neater way to do this...?
for item in xeProps.iterrows():
    # print(item)
    
    # Name from index - this 
    # itemInds = ''.join((str(value) for value in item[0]))
    itemInds = [''.join(str(value)) for value in item[0]]
    itemName = '_'.join(itemInds).replace('.','')  # Set param name from indexes
                                                   # Note "[a-z_][a-z0-9_]*" and valid python var name.
    
    # print(itemName)
    # print(type(itemName))
    # print((''.join(str(value)) for value in item[0]))
    # list(item[0]).join('')
    # params.add(f"Splitting_{itemName}", value = item[-1][0], min = 0, max = 1)
    
    # OK up to itemName[0:5]
    # params.add(f"s_{itemName[0:5]}", value = item[-1][0], min = 0, max = 1)
    # params.add(f"s{itemName[0:7]}", value = item[-1][0], min = 0, max = 1)
    
    # print(item[-1])  #['Splitting/cm−1'])
    itemVal = unumpy.nominal_values(item[-1]['Splitting/cm−1'])
    params.add(f"s_{itemName}", value = itemVal, min = 0, max = 1)

# params.add('amp', value=10, vary=False)
# params.add('decay', value=0.007, min=0.0)
# params.add('phase', value=0.2)
# params.add('frequency', value=3.0, max=10)
                            
params

# TODO: easy/neat way to index these back to original case... see PEMtk....
# UPDATE: main difference there is piece-wise list/dict creation prior to setting params, but quite similar.

name,value,initial value,min,max,vary
s_129_05_05_15,0.28630000,0.2863,0.00000000,1.00000000,True
s_131_15_15_05,0.08550000,0.0855,0.00000000,1.00000000,True
s_131_15_25_15,0.14110000,0.1411,0.00000000,1.00000000,True
s_131_15_25_05,0.22760000,0.2276,0.00000000,1.00000000,True


In [108]:
# Map items to names
xeProps.index.to_list()

In [98]:
params.keys()

dict_keys(['s_129_05_05_15', 's_131_15_15_05', 's_131_15_25_15', 's_131_15_25_05'])

In [104]:
params.valuesdict()

{'s_129_05_05_15': array(0.2863),
 's_131_15_15_05': array(0.0855),
 's_131_15_25_15': array(0.1411),
 's_131_15_25_05': array(0.2276)}

In [4]:
def testLocals(**kwargs):
    
    z=2
    
    return locals()

testLocals(x=5)

{'kwargs': {'x': 5}, 'z': 2}

## lmfit reboot/test

From https://lmfit.github.io/lmfit-py/intro.html

In [24]:
from numpy import linspace, random
from scipy.optimize import leastsq

# generate synthetic data with noise
x = linspace(0, 100)
noise = random.normal(size=x.size, scale=0.2)
data = 7.5 * sin(x*0.22 + 2.5) * exp(-x*x*0.01) + noise

# generate experimental uncertainties
uncertainty = abs(0.16 + random.normal(size=x.size, scale=0.05))

In [25]:
from numpy import exp, sin

from lmfit import minimize, Parameters


def residual(params, x, data, uncertainty):
    amp = params['amp']
    phaseshift = params['phase']
    freq = params['frequency']
    decay = params['decay']

    model = amp * sin(x*freq + phaseshift) * exp(-x*x*decay)

    return (data-model) / uncertainty


params = Parameters()
params.add('amp', value=10)
params.add('decay', value=0.007)
params.add('phase', value=0.2)
params.add('frequency', value=3.0)

out = minimize(residual, params, args=(x, data, uncertainty))

In [26]:
out

## Test some PEMtk methods...

In [105]:
from pemtk.fit._util import lmmuListStrReformat

lmmuListStrReformat(xeProps.index)

['129_0.5_0.5_1.5', '131_1.5_1.5_0.5', '131_1.5_2.5_1.5', '131_1.5_2.5_0.5']